In [42]:
!pip uninstall -y numpy
!pip install -q "numpy<2" ultralytics kaggle

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.5 which is incompatible.
google-colab 1.0.0 requires tornado==6.4.2, but you have tornado 6.5.2 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have g

In [43]:
import numpy as np
print(f"NumPy version: {np.__version__}")

NumPy version: 1.26.4


In [44]:
import os
import torch
from ultralytics import YOLO
from pathlib import Path
import shutil

In [45]:
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPUs: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA: True
GPUs: 2
GPU 0: Tesla T4
GPU 1: Tesla T4


In [46]:
os.environ['KAGGLE_USERNAME'] = 'kaggle'  # автоматически в Kaggle
os.environ['KAGGLE_KEY'] = 'kaggle'

# Создаём рабочую директорию
!mkdir -p /kaggle/working/dataset


In [47]:
print("📥 Скачивание датасета LISA Traffic Light...")

!kaggle datasets download -d kietzer0/lisa-traffic-light-dataset-yolo -p /kaggle/working/dataset --unzip

print("✓ Датасет скачан!")

📥 Скачивание датасета LISA Traffic Light...
Dataset URL: https://www.kaggle.com/datasets/kietzer0/lisa-traffic-light-dataset-yolo
License(s): CC-BY-NC-SA-4.0
 98%|█████████████████████████████████████ | 2.33G/2.39G [00:01<00:00, 1.49GB/s]
100%|██████████████████████████████████████| 2.39G/2.39G [00:01<00:00, 1.47GB/s]
✓ Датасет скачан!


In [48]:
dataset_path = "/kaggle/working/dataset"

print("\n📂 Полная структура датасета:")
!find {dataset_path} -type d | head -20

print("\n🔍 Поиск папок с изображениями:")
!find {dataset_path} -type d -name "images" -o -name "train" -o -name "valid" -o -name "test"


📂 Полная структура датасета:
/kaggle/working/dataset
/kaggle/working/dataset/datasets
/kaggle/working/dataset/datasets/test
/kaggle/working/dataset/datasets/test/labels
/kaggle/working/dataset/datasets/test/images
/kaggle/working/dataset/datasets/val
/kaggle/working/dataset/datasets/val/labels
/kaggle/working/dataset/datasets/val/images
/kaggle/working/dataset/datasets/train
/kaggle/working/dataset/datasets/train/labels
/kaggle/working/dataset/datasets/train/images

🔍 Поиск папок с изображениями:
/kaggle/working/dataset/datasets/test
/kaggle/working/dataset/datasets/test/images
/kaggle/working/dataset/datasets/val/images
/kaggle/working/dataset/datasets/train
/kaggle/working/dataset/datasets/train/images


In [49]:
import glob

def find_dataset_structure(base_path):
    """Находит правильную структуру датасета"""
    
    # Возможные варианты структуры
    patterns = [
        # Вариант 1: train/images, valid/images
        {'train': 'train/images', 'val': 'valid/images', 'test': 'test/images'},
        # Вариант 2: images/train, images/valid
        {'train': 'images/train', 'val': 'images/valid', 'test': 'images/test'},
        # Вариант 3: Train, Valid, Test (с заглавными)
        {'train': 'Train', 'val': 'Valid', 'test': 'Test'},
        # Вариант 4: train, valid, test (прямо в корне)
        {'train': 'train', 'val': 'valid', 'test': 'test'},
        {'train': 'train', 'val': 'val', 'test': 'test'},
    ]
    
    for pattern in patterns:
        train_path = os.path.join(base_path, pattern['train'])
        if os.path.exists(train_path):
            # Проверяем есть ли изображения
            images = glob.glob(f"{train_path}/*.jpg") + glob.glob(f"{train_path}/*.png")
            if images:
                print(f"✓ Найдена структура: {pattern}")
                return pattern
    
    # Если не нашли, ищем любую папку с images
    all_image_dirs = []
    for root, dirs, files in os.walk(base_path):
        if any(f.endswith(('.jpg', '.png')) for f in files):
            rel_path = os.path.relpath(root, base_path)
            all_image_dirs.append(rel_path)
    
    if all_image_dirs:
        print(f"✓ Найдены папки: {all_image_dirs[:5]}")
        train_dir = [d for d in all_image_dirs if 'train' in d.lower()]
        val_dir = [d for d in all_image_dirs if 'valid' in d.lower() or 'val' in d.lower()]
        
        if train_dir and val_dir:
            return {'train': train_dir[0], 'val': val_dir[0], 'test': val_dir[0]}
    
    return None

# Определяем структуру
structure = find_dataset_structure(dataset_path)

if structure is None:
    print("❌ Не удалось определить структуру!")
    !ls -R {dataset_path} | head -50
else:
    print(f"\n✓ Структура датасета:")
    for key, val in structure.items():
        full_path = os.path.join(dataset_path, val)
        img_count = len(glob.glob(f"{full_path}/*.jpg") + glob.glob(f"{full_path}/*.png"))
        print(f"  {key:10s}: {val:30s} ({img_count} images)")

✓ Найдены папки: ['datasets/test/images', 'datasets/val/images', 'datasets/train/images']

✓ Структура датасета:
  train     : datasets/train/images          (12775 images)
  val       : datasets/val/images            (1277 images)
  test      : datasets/val/images            (1277 images)


In [59]:
# %% ЗАМЕНИТЬ ЯЧЕЙКУ С СОЗДАНИЕМ YAML НА ЭТУ:

# Создаём правильный data.yaml
yaml_path = "/kaggle/working/data.yaml"

yaml_content = f"""path: {dataset_path}
train: {structure['train']}
val: {structure['val']}

nc: 1
names: ['traffic_light']
"""

with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"✓ Создан data.yaml:")
print(yaml_content)

# Проверка путей
print("\n🔍 Проверка:")
train_full = os.path.join(dataset_path, structure['train'])
val_full = os.path.join(dataset_path, structure['val'])

print(f"Train exists: {os.path.exists(train_full)}")
print(f"Val exists: {os.path.exists(val_full)}")

train_imgs = glob.glob(f"{train_full}/*.jpg") + glob.glob(f"{train_full}/*.png")
val_imgs = glob.glob(f"{val_full}/*.jpg") + glob.glob(f"{val_full}/*.png")

print(f"Train images: {len(train_imgs)}")
print(f"Val images: {len(val_imgs)}")

if len(train_imgs) == 0:
    print("❌ ОШИБКА: Нет изображений в train!")
elif len(val_imgs) == 0:
    print("❌ ОШИБКА: Нет изображений в val!")
else:
    print("✅ Датасет готов к обучению!")


✓ Создан data.yaml:
path: /kaggle/working/dataset
train: datasets/train/images
val: datasets/val/images

nc: 1
names: ['traffic_light']


🔍 Проверка:
Train exists: True
Val exists: True
Train images: 12775
Val images: 1277
✅ Датасет готов к обучению!


In [60]:
# Подсчёт изображений
print("\n📊 Статистика датасета:")
train_images = !find {dataset_path} -path "*/train/images/*" -o -path "*/images/train/*" | wc -l
val_images = !find {dataset_path} -path "*/valid/images/*" -o -path "*/val/images/*" -o -path "*/images/val/*" | wc -l

print(f"Train: {train_images[0] if train_images else 0} изображений")
print(f"Val: {val_images[0] if val_images else 0} изображений")


📊 Статистика датасета:
Train: 12775 изображений
Val: 1277 изображений


In [61]:
model = YOLO('yolo11n.pt')

In [62]:
# %%
print("🔍 Проверка сохранённых весов:")
!ls -lh /kaggle/working/runs/traffic_light/weights/

print("\n📊 Сколько эпох завершилось:")
!ls /kaggle/working/runs/traffic_light/weights/epoch*.pt 2>/dev/null | wc -l

print("\n✅ Best и Last веса:")
!ls -lh /kaggle/working/runs/traffic_light/weights/best.pt 2>/dev/null || echo "❌ best.pt не найден"
!ls -lh /kaggle/working/runs/traffic_light/weights/last.pt 2>/dev/null || echo "❌ last.pt не найден"


🔍 Проверка сохранённых весов:
total 138M
-rw-r--r-- 1 root root 16M Dec  5 16:32 best.pt
-rw-r--r-- 1 root root 16M Dec  5 16:19 epoch0.pt
-rw-r--r-- 1 root root 16M Dec  5 16:22 epoch1.pt
-rw-r--r-- 1 root root 16M Dec  5 16:24 epoch2.pt
-rw-r--r-- 1 root root 16M Dec  5 16:27 epoch3.pt
-rw-r--r-- 1 root root 16M Dec  5 16:29 epoch4.pt
-rw-r--r-- 1 root root 16M Dec  5 16:32 epoch5.pt
-rw-r--r-- 1 root root 16M Dec  5 16:34 epoch6.pt
-rw-r--r-- 1 root root 16M Dec  5 16:34 last.pt

📊 Сколько эпох завершилось:
7

✅ Best и Last веса:
-rw-r--r-- 1 root root 16M Dec  5 16:32 /kaggle/working/runs/traffic_light/weights/best.pt
-rw-r--r-- 1 root root 16M Dec  5 16:34 /kaggle/working/runs/traffic_light/weights/last.pt


In [63]:
results = model.train(
    data=yaml_path,
    epochs=20,
    imgsz=640,
    batch=32,                    # 16 на каждую GPU
    device='0,1',                # 2x T4
    name='traffic_light',
    project='/kaggle/working/runs',
    
    # Скорость
    workers=8,
    cache=True,
    
    # Аугментации
    augment=True,
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
    
    # Оптимизатор
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    
    # Сохранение
    save=True,
    save_period=1,
    plots=True,
    
    # БЕЗ валидации
    val=True,
    
    verbose=True,
    exist_ok=True
)


Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=traffic_light, nbs=64, nms=False, opset=None, optimize=False,

In [58]:
from collections import Counter

cls_counts = Counter()
for f in labels:
    for line in open(f):
        if not line.strip():
            continue
        c = int(line.split()[0])
        cls_counts[c] += 1
print(cls_counts)


Counter({0: 49517})


In [64]:
output_dir = Path('/kaggle/working/final_weights')
output_dir.mkdir(exist_ok=True)

# Копируем веса
weights_dir = Path('/kaggle/working/runs/traffic_light/weights')

shutil.copy(weights_dir / 'best.pt', output_dir / 'best.pt')
shutil.copy(weights_dir / 'last.pt', output_dir / 'last.pt')

# Копируем графики
results_dir = Path('/kaggle/working/runs/traffic_light')
if (results_dir / 'results.png').exists():
    shutil.copy(results_dir / 'results.png', output_dir / 'training_curves.png')
if (results_dir / 'confusion_matrix.png').exists():
    shutil.copy(results_dir / 'confusion_matrix.png', output_dir / 'confusion_matrix.png')